In [ ]:
### Investment portfolio in Python ###
## Obtaining the portfolio ##

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates # Required to style dates
# Necessary for when working on notebooks
# %matplotlib inline 

import datetime as dt # Used to define dates and time

import time

import yfinance as yf # Used to download stock information from Yahoo Finance
import os # To work with directories and files in the operating system
from os import listdir
from os.path import isfile, join

import cufflinks as cf # Library for connecting plotly with pandas
import plotly.express as px
import plotly.graph_objects as go

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected = True)

cf.go_offline()

from plotly.subplots import make_subplots

import warnings
warnings.simplefilter('ignore')


#%%
# Variable definition
PATH = 'C:\Users\Felipe\Desktop\PortafoliodeInversionPython\CSV\Wilshire/'

# Default start and end dates
S_DATE = '2017-02-01'
E_DATE = '2022-06-19'
S_DATE_DT = pd.to_datetime(S_DATE)
E_DATE_DT = pd.to_datetime(E_DATE)


risk_free_rate = 0.0125 # Approximately the 10-year bond rate
#%%
# Getting the file names in a list

files = [x for x in listdir(PATH) if isfile(join(PATH, x))]
tickers = [os.path.splitext(x)[0] for x in files]
tickers
tickers.sort()
len(tickers)

#%%
# Get the data from the created CSVs

def get_stock_df_from_csv(ticker):
    try:
        df = pd.read_csv(PATH + ticker + '.csv', index_col=0)
    except FileNotFoundError:
        print('The file does not exists')
    else:
        return df
    
#%%
# Join multiple actions by column name into a single df

def merge_df_by_column_name(col_name, sdate, edate, *tickers):
    # Will hold data for all dataframes with the same column name
    mult_df = pd.DataFrame()
    
    for x in tickers:
        df = get_stock_df_from_csv(x)
        mask = (df.index >= sdate) & (df.index <= edate)
        mult_df[x] = df.loc[mask][col_name]
        
    return mult_df

#%%
# Markowitz Portfolio Optimization

"""
Harry Markowitz proved that you could make what is called an efficient portfolio. That is a portfolio that optimizes return while also minimizing risk. We don't benefit from analyzing individual securities at the same rate as if we instead considered a portfolio of stocks.
We do this by creating portfolios with stocks that are not correlated. We want to calculate expected returns by analyzing the returns of each stock multiplied by its weight.
w1r1 + w2r2 = rp
The standard deviation of the portfolio is found this way. Sum multiple calculations starting by finding the product of the first securities weight squared times its standard deviation squared. The middle is 2 times the correlation coefficient between the stocks. 
And, finally add those to the weight squared times the standard deviation squared for the second security.
(w1d1 + w2d2)^2 = w1^2*d1^2 + 2w1d1w2d2 + w2^2 * d2^2
"""

# Draw the most efficient frontier
# Selection of a portfolio with shares previously studied, with the data of the accumulated performance and Ishimoku
# For that matter I selected some of specific sectors.

port_list = ['PLUG', 'AMRC', 'GNRC',
'HCC', 'RFP', 'CF',
'IIPR', 'BRT', 'BRG',
'CDNA', 'ZYXI', 'ARWR',
'ATLC', 'KNSL', 'LPLA',
'ENPH', 'APPS', 'SEDG',
'RCMT', 'FCN', 'MHH',
'NEE', 'MSEX', 'EXC',
'TTGT', 'ROKU', 'IRDM',
'OAS', 'VTNR', 'EGY']

num_stocks = len(port_list)
print(num_stocks)

# Generate a df with the closing prices of all selected stocks

mult_df = merge_df_by_column_name('Close', S_DATE, E_DATE, *port_list)

#%%
# Generate a chart for stock prices

fig = px.line(mult_df, x = mult_df.index, y = mult_df.columns)
fig.update_layout(height=1000, width=1800, showlegend=True)
fig.update_xaxes(title="Date", rangeslider_visible=True)
fig.update_yaxes(title="Price")
plot(fig)

#%%
# Generate a price transformation and chart

mult_df_t = np.log10(mult_df)

fig = px.line(mult_df_t, x = mult_df_t.index, y = mult_df_t.columns)
fig.update_layout(height=1000, width=1800, showlegend=True)
fig.update_xaxes(title="Date", rangeslider_visible=True)
fig.update_yaxes(title="Log10 Price")
plot(fig)

#%%
# Average returns for one year (252 business days)

returns = np.log(mult_df / mult_df.shift(1))
mean_ret = returns.mean()*252
print(mean_ret)

#%%
# Calculation of the correlation of actions
returns.corr()

# Stock Correlation Chart
# We want a portfolio with low correlation between stocks
import seaborn as sns
correlation_matrix = returns.corr(method='spearman')
fig = sns.heatmap(correlation_matrix, annot=False)
# fig.update_layout(height=1000, width=1800, showlegend=True)
plt.show()

#%%
# Generation of random weights whose sum is one

weights = np.random.random(num_stocks)
weights /= np.sum(weights)  # weights = weights / np.sum(weights)
print('Weights: ', weights)
print('Total weight: ', np.sum(weights))

# Calculation of the average annual return with the random weights
print(np.sum(weights * returns.mean()) * 252)

#%%
# Volatility Calculation
# Portfolio risk with current weights

print(np.sqrt(np.dot(weights.T, np.dot(returns.cov() * 252, weights))))

#%%
# Running a simulation of 10000 portfolios using a function

p_ret = [] # Returns list
p_vol = [] # Volatility list
p_SR = [] # Sharpe Ratio list
p_wt = [] # Weights per portfolio list


for x in range(10000):
    # Generate randoms weights
    p_weights = np.random.random(num_stocks)
    p_weights /= np.sum(p_weights)
    
    # Calculation of the return according to the weights
    ret_1 = np.sum(p_weights * returns.mean()) * 252
    p_ret.append(ret_1)
    
    # Volatility Calculation
    vol_1 = np.sqrt(np.dot(p_weights.T, np.dot(returns.cov() * 252, p_weights)))
    p_vol.append(vol_1)
    
    # Calculation of the Sharpe ratio
    SR_1 = (ret_1 - risk_free_rate) / vol_1
    p_SR.append(SR_1)
    
    # Store the weights for each portfolio
    p_wt.append(p_weights)
    
# Convert to numpy arrays
p_ret = np.array(p_ret)
p_vol = np.array(p_vol)
p_SR = np.array(p_SR)
p_wt = np.array(p_wt)

p_ret, p_vol, p_SR, p_wt

#%%
# Graph of the simulated portfolios or most efficient frontier
ports = pd.DataFrame({'Returns': p_ret, 'Volatility': p_vol, })
ports.plot(x='Volatility', y = 'Returns', kind = 'scatter', figsize = (19,9))

#%%
# Sharpe ratio

"""
People want to maximize returns while avoiding as much risk as possible. 
William Sharpe created the Sharpe Ratio to find the portfolio that provides the best return for the lowest amount of risk.
As return increases so does the Sharpe Ratio, but as Standard Deviation increase the Sharpe Ration decreases.
"""
# Returns the index for the highest Sharpe Ratio
SR_idx = np.argmax(p_SR)

# Find the ideal weights for the portfolio in that index
i = 0
while i < num_stocks:
    print("Stock : %s : %2.2f" % (port_list[i], (p_wt[SR_idx][i] * 100)))
    i += 1
    
# Find the volatility of that portfolio
print("\nVolatility :", p_vol[SR_idx] * 100)
      
# Find the return on that portfolio
print("Return :", p_ret[SR_idx] * 100)

# You can also take percentages less than one and bring them closer to one, then calculate the portfolio.
# In situations in which the percentages are less than one, what can be done is to bring them closer to one or to an action, or directly discard them.